## Week 2 Day 3 - Mailjet replacemnt of sendgrid


see mailjet.com for getting an account and public and private keys
pip install mailjet_rest

Now we get to more detail:

1. Different models
    DEEPSEEK
    GROQ LAMA
    GOOGLE
    OPENAI

2. Use Mailjet.com for sending emails

In [ ]:
!pip install mailjet_rest
from mailjet_rest import Client
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os

In [ ]:
load_dotenv(override=True)

In [ ]:
# mailjet replacement code

load_dotenv(override=True)

if not os.getenv("OPENAI_API_KEY"):
    print("Error: OPENAI_API_KEY not found!")
    print("Please set your OpenAI API key in a .env file")
else:
    print("Mailjet API key found! Ready to generate mail.")

if not os.getenv("MJ_APIKEY_PUBLIC"):
    print("Error: MJ_APIKEY_PUBLIC not found!")
    print("Please set your mj api key in a .env file")
else:
    api_key = os.environ["MJ_APIKEY_PUBLIC"]
    print(f"API key {api_key} found! Ready to generate mail`.")

if not os.getenv("MJ_APIKEY_PRIVATE"):
    print("Error: MJ_APIKEY_PRIVATE not found!")
    print("Please set your mj api key in a .env file")
else:
    api_secret = os.environ["MJ_APIKEY_PRIVATE"]
    print(f"API decret {api_secret} key found! Ready to generate images.")
# api_secret = os.environ["MJ_APIKEY_PRIVATE"]
from_email=os.environ["FROM_EMAIL"]
to_email=os.environ["TO_EMAIL"]
print(f"from email {from_email} , to email {to_email} ")


In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

instructions3 = "You are a detailed sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [ ]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
open_ai_client = AsyncOpenAI(api_key=openai_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
open_ai_model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=open_ai_client)

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

instructions4 = "You are a detailed sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3 =  Agent(name="Llama3.3 Sales Agent", instructions=instructions3, model=llama3_3_model)
sales_agent4 =  Agent(name="OpenAI Sales Agent", instructions=instructions4, model=open_ai_model)

# sales_picker = Agent(name="Sales Picker", instructions=instructions5, model=open_ai_model)

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)
tool4 = sales_agent4.as_tool(tool_name="sales_agent4", tool_description=description)

In [ ]:
# modify send email

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    #Send out an email useing mailjet
    print(f"preparing to email {from_email} , {to_email}, {subject}")
    mailjet = Client(auth=(api_key, api_secret))
    data = {
        "FromEmail": from_email,
        "FromName":"Email agent - Sales pircker",
        "Subject": subject,
        "Text-part": "Dear passenger, welcome to Mailjet! May the delivery force be with you!",
        "Html-part": html_body,
        "Recipients": [{"Email": to_email}]
         }
    result = mailjet.send.create(data=data)
    print("mailjet status code ",result.status_code)
    print(result.json())
    #  push(f"sent email status = {result.status_code} f= ({from_email} ,t = {to_email}")
    # push("email end")
    return {"status": "success"}

In [ ]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [ ]:
email_tools = [subject_tool, html_tool, send_html_email]

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
# tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
#m out of Deepseek agent1 credits so replace with openai agent4
tools = [tool2, tool3, tool4]

In [ ]:
print(tools)

print(handoffs)

In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 2 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from the Compli yeahsales manager"

with trace("Automated SDR with Mailjet.com for email sending"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces